In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd "/content/drive/MyDrive/hj/nlp/"

/content/drive/MyDrive/hj/nlp


In [3]:
!pip install pytorch-pretrained-bert
!pip install zstandard
!pip install flashtext
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 123 kB 19.5 MB/s 
     |████████████████████████████████| 132 kB 66.9 MB/s 
     |████████████████████████████████| 10.3 MB 60.7 MB/s 
     |████████████████████████████████| 79 kB 9.3 MB/s 
     |████████████████████████████████| 140 kB 69.9 MB/s 
     |████████████████████████████████| 127 kB 70.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.5 MB 11.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9307 sha256=e08108c15a47807fc785e42f6e2e2e9d0a6a4c0521ebe65

## git clone

In [ ]:
!git clone https://github.com/microsoft/DialoGPT.git

Cloning into 'DialoGPT'...
remote: Enumerating objects: 344, done.
remote: Counting objects: 100% (175/175), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 344 (delta 131), reused 132 (delta 125), pack-reused 169
Receiving objects: 100% (344/344), 44.82 MiB | 14.03 MiB/s, done.
Resolving deltas: 100% (160/160), done.


In [ ]:
!git clone https://github.com/andreamad8/DialoGPT2-Interact.git

Cloning into 'DialoGPT2-Interact'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 15 (delta 2), reused 3 (delta 0), pack-reused 0
Unpacking objects: 100% (15/15), done.


In [4]:
cd DialoGPT

/content/drive/MyDrive/hj/nlp/DialoGPT


## 데이터 준비

In [ ]:
!python interact.py --model_name_or_path ./models/medium --load_checkpoint ./models/medium/medium_ft.pkl --top_k 10

Traceback (most recent call last):
  File "interact.py", line 7, in <module>
    import torch
  File "/usr/local/lib/python3.8/dist-packages/torch/__init__.py", line 649, in <module>
    from ._tensor import Tensor
  File "/usr/local/lib/python3.8/dist-packages/torch/_tensor.py", line 21, in <module>
    from torch.overrides import (
  File "/usr/local/lib/python3.8/dist-packages/torch/overrides.py", line 54, in <module>
    def get_ignored_functions() -> Set[Callable]:
  File "/usr/lib/python3.8/typing.py", line 258, in inner
    return cached(*args, **kwds)
  File "/usr/lib/python3.8/typing.py", line 687, in __getitem__
    return _subs_tvars(self, self.__parameters__, params)
  File "/usr/lib/python3.8/typing.py", line 203, in _subs_tvars
    return tp.copy_with(tuple(new_args))
  File "/usr/lib/python3.8/typing.py", line 691, in copy_with
    return _GenericAlias(self.__origin__, params, name=self._name, inst=self._inst)
  File "/usr/lib/python3.8/typing.py", line 669, in __init__


In [ ]:
!python demo.py --data small

PROJECT_FOLDER = /content/drive/MyDrive/hj/nlp/DialoGPT
Found existing models folder at /content/drive/MyDrive/hj/nlp/DialoGPT/models, skip creating a new one!
12/28/2022 18:02:52 - INFO - __main__ -   Downloading models...
100% 176/176 [00:00<00:00, 187074.89B/s]
100% 1042301/1042301 [00:01<00:00, 571830.22B/s]
100% 456318/456318 [00:01<00:00, 323244.65B/s]
100% 548118077/548118077 [01:47<00:00, 5101282.44B/s]
100% 351265273/351265273 [01:31<00:00, 3851744.56B/s]
12/28/2022 18:06:18 - INFO - __main__ -   Done!

12/28/2022 18:06:18 - INFO - __main__ -   Downloading and Extracting Data...
12/28/2022 18:24:03 - INFO - __main__ -   Preparing Data...
prepro.py --corpus /content/drive/MyDrive/hj/nlp/DialoGPT/data/train.tsv --max_seq_len 128
12/28/2022 18:28:31 - INFO - __main__ -   Done!

12/28/2022 18:28:31 - INFO - __main__ -   Generating training CMD!
12/28/2022 18:28:31 - INFO - __main__ -   If there is any problem, please copy (modify) and run command below
12/28/2022 18:28:31 - INFO -

## 패키지 준비



In [5]:
import json
from os.path import abspath, dirname, exists, join
import argparse
import logging
from tqdm import trange
import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import numpy as np
import socket
import os, sys
import re
import logging
from functools import partial
#from demo_utils import download_model_folder
import argparse
import subprocess as sp
from tqdm import tqdm

from data_loader import BucketingDataLoader, DynamicBatchingLoader, DistributedBucketingDataLoader
#from pytorch_pretrained_bert import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config, BertForSequenceClassification, BertConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from gpt2_training.train_utils import get_eval_list_same_length, load_model, boolean_string, fix_state_dict_namespace

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)


EOS_ID = 50256

## 함수 정의

In [6]:
def cut_seq_to_eos(sentence, remove_id=[-1]):
    sent=[]
    for s in sentence:
        if s in remove_id:
            continue
        if s != EOS_ID:
            sent.append(s)
        else:
            break
    return sent


### FROM HUGGING FACE REPO
def top_filtering(logits, top_k=0, top_p=0.0, threshold=-float('Inf'), filter_value=-float('Inf')):
    """ Filter a distribution of logits using top-k, top-p (nucleus) and/or threshold filtering
        Args:
            logits: logits distribution shape (vocabulary size)
            top_k: <=0: no filtering, >0: keep only top k tokens with highest probability.
            top_p: <=0.0: no filtering, >0.0: keep only a subset S of candidates, where S is the smallest subset
                whose total probability mass is greater than or equal to the threshold top_p.
                In practice, we select the highest probability tokens whose cumulative probability mass exceeds
                the threshold top_p.
            threshold: a minimal threshold to keep logits
    """
    assert logits.dim() == 1  # Only work for batch size 1 for now - could update but it would obfuscate a bit the code
    top_k = min(top_k, logits.size(-1))
    if top_k > 0:
        # Remove all tokens with a probability less than the last token in the top-k tokens
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        # Compute cumulative probabilities of sorted tokens
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probabilities = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probabilities > top_p
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        # Back to unsorted indices and set them to -infinity
        indices_to_remove = sorted_indices[sorted_indices_to_remove]
        logits[indices_to_remove] = filter_value

    indices_to_remove = logits < threshold
    logits[indices_to_remove] = filter_value
    return logits

def generate_next_token(model, input_ids, position_ids=None, token_type_ids=None, prev=None, temperature=1, top_k=0, top_p=0, past=None):
    with torch.no_grad():
        if not past:
            hidden_states, past = model.transformer(prev, position_ids, token_type_ids, past)
        else:
            hidden_states, past = model.transformer(prev, past=past)
        logits = model.lm_head(hidden_states)
        logits = logits[0, -1, :] / temperature
        logits = top_filtering(logits, top_k=top_k, top_p=top_p)
        probs = F.softmax(logits.unsqueeze(0), dim=-1)
        prev = torch.multinomial(probs, num_samples=1)
        return prev, probs[0][prev], past

def generate_sequence(model, input_ids, position_ids=None, token_type_ids=None, temperature=1, top_k=0, top_p=0, length=20, past=None, device='cuda'):
    output = input_ids.new_zeros([input_ids.size(0),0])
    prev = input_ids
    for i in range(length):
        prev, probs, past = generate_next_token(model, input_ids, position_ids, token_type_ids, prev, temperature, top_k, top_p, past)
        output = torch.cat((output, prev), dim=1)
    return output

def cut_seq_to_eos(sentence, remove_id=[-1]):
    sent=[]
    for s in sentence:
        if s in remove_id:
            continue
        if s != EOS_ID:
            sent.append(s)
        else:
            break
    return sent

## 데이터에 대해 응답 생성, 출력

In [7]:
parser = argparse.ArgumentParser()
parser.add_argument('--model_name_or_path', type=str, default='./models/medium', help='pretrained model name or path to local checkpoint')
parser.add_argument("--seed", type=int, default=42)
parser.add_argument("--load_checkpoint", '-c', type=str, default='./models/medium/medium_ft.pkl')
parser.add_argument("--fp16", type=boolean_string, default=False)
parser.add_argument("--max_seq_length", type=int, default=128)
parser.add_argument("--train_batch_size", type=int, default=1,
                    help="batch size now means per GPU per step")
parser.add_argument("--train_input_file", type=str, default='./data/train.128len.db')
    
parser.add_argument("--generation_length", type=int, default=20)
parser.add_argument("--max_history", type=int, default=2)

parser.add_argument("--temperature", type=float, default=1)
parser.add_argument("--top_k", type=int, default=10)
parser.add_argument("--top_p", type=float, default=0.9)

parser.add_argument('--use_gpu', action='store_true')
parser.add_argument("--gpu", type=int, default=0)
parser.add_argument("-f", type=str)

args = parser.parse_args()
os.environ['CUDA_VISIBLE_DEVICES'] = str(args.gpu)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
args.device, args.n_gpu = device, n_gpu

np.random.seed(42)
torch.random.manual_seed(42)
torch.cuda.manual_seed(42)

#### load the GPT-2 model 
#config = GPT2Config.from_json_file(os.path.join(args.model_name_or_path, 'config.json'))
#enc = GPT2Tokenizer.from_pretrained(args.model_name_or_path)
#model = load_model(GPT2LMHeadModel(config), args.load_checkpoint, args, verbose=True)
model_name = "microsoft/DialoGPT-medium"
enc = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.to(device)
model.eval()

#########################################################################
# Prepare Data Set
##########################################################################

train_dataloader = BucketingDataLoader(args.train_input_file,
                                        args.train_batch_size,
                                        args.max_seq_length)

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/863M [00:00<?, ?B/s]

In [8]:
i = 0
model.eval()
for batch in train_dataloader:
    if i==3:
        break
    #print(input_ids.shape, position_ids.shape, token_ids.shape, label_ids.shape)
    batch = tuple(t.to(device) for t in batch)
    input_ids, position_ids, token_ids, label_ids, *_ = batch
    input_ids = input_ids[0].reshape(1, -1)
    label_ids = label_ids[0].reshape(1, -1)
    
    input_ids = torch.cat((torch.tensor(cut_seq_to_eos(input_ids[0].tolist())).reshape(1, -1), torch.tensor([[EOS_ID]])), dim=1).to(device)

    out = model.generate(input_ids, pad_token_id=-1, max_length=1000, eos_token_id=EOS_ID, num_beams=5, num_return_sequences=5)
    
    input_ids = input_ids[0][:-1]
    label_ids = label_ids[label_ids!=-1][:-1].tolist()
    label_ids = cut_seq_to_eos(label_ids)

    print("Input : ", enc.decode(input_ids))
    for j in range(5):
        output = out[j]
        output = output[output!=-1][input_ids.size(-1)+1:-1]
        print("Output{} : ".format(j+1), enc.decode(output))
    print("Label : ", enc.decode(label_ids))
    print("\n")

    i+=1

Input :  Can it go negative?
Output1 :  It can't go negative.
Output2 :  It can go negative, but it can't go negative.
Output3 :  It can go negative, but it can't go positive.
Output4 :  It can go negative, but it won't go negative.
Output5 :  No, but it can go positive.
Label :  If it does, will it signify that people also disapprove of the job he's not doing?


Input :  It's high comedy! A morbidly obese man making a documentary about health care? C'mon! Laugh along.
Output1 :  It's high comedy! A morbidly obese man making a documentary about health care? C'mon! Laugh along.
Output2 :  It's high comedy! A morbidly obese man making a documentary about health care? C'mon! Laugh along. english haiku bot
Output3 :  It's high comedy! A morbidly obese man making a documentary about health care? C'mon! Laugh along. FTFY
Output4 :  It's high comedy! A morbidly obese man making a documentary about health care? C'mon! laugh along.
Output5 :  I laughed.
Label :  he actually lost a lot of weight

## Discriminator 모델 정의

In [9]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.num_layers = 10
        self.lstm = nn.LSTM(input_size=1, hidden_size=512, num_layers=self.num_layers, batch_first=True, bidirectional=True, dropout=0.3)
        self.fc1 = nn.Linear(512*2*self.num_layers, 256)
        self.fc2 = nn.Linear(256, 1)
        self.leakyrelu = nn.LeakyReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, input):
        batch_size = input.size(0)
        h0 = torch.zeros((2*self.num_layers, batch_size, 512)).to(device)
        c0 = torch.zeros((2*self.num_layers, batch_size, 512)).to(device)
        input = input.reshape(batch_size, -1, 1).float()
        _, (h, c) = self.lstm(input, (h0, c0))
        h = h.reshape(batch_size, 1024*self.num_layers)
        output = self.leakyrelu(self.fc1(self.leakyrelu(h)))
        output = self.sigmoid(self.fc2(output))

        return output

In [10]:
torch.backends.cudnn.enabled = False
D = Discriminator().to(device)
#D = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1).to(device)
criterion = nn.BCELoss()
optimizer = Adam(D.parameters(), lr=0.001)

D.load_state_dict(torch.load('D_lstm10_fc2.pkl'))

<All keys matched successfully>

## 작은 데이터셋 생성


In [11]:
num_train_samples = 10000
num_test_samples = 1000
train_dataset = []
test_dataset = []

for i, batch in enumerate(train_dataloader):
    if i < num_train_samples:
        train_dataset.append(batch)
    if i >= num_train_samples and i < num_train_samples+num_test_samples:
        test_dataset.append(batch)
    if i == num_train_samples+num_test_samples:
        break
    if i % 100 == 0:
        print(i)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900


## Discriminator 학습

In [ ]:
out_real = []
out_fake = []
for epoch in range(10):
  i = 0
  for batch in train_dataset:
      #print(input_ids.shape, position_ids.shape, token_ids.shape, label_ids.shape)
      batch = tuple(t.to(device) for t in batch)
      input_ids, position_ids, token_ids, label_ids, *_ = batch
      input_ids = input_ids[0].reshape(1, -1)
      label_ids = label_ids[0].reshape(1, -1)
      
      #모델 출력 생성 (out)
      input_ids = torch.cat((torch.tensor(cut_seq_to_eos(input_ids[0].tolist())).reshape(1, -1), torch.tensor([[EOS_ID]])), dim=1).to(device).long()

      out = model.generate(input_ids, pad_token_id=-1, max_length=1000, eos_token_id=EOS_ID, num_beams=5, num_return_sequences=1)
      out = out[0][input_ids.size(-1)+1:-1].reshape(1, -1)

      label_ids = label_ids[label_ids!=-1]
      label_ids = torch.tensor(cut_seq_to_eos(label_ids.tolist())).reshape(1, -1).to(device)

      #discriminator 입력 생성 (질문과 응답을 이어붙임)
      real_input = torch.cat((input_ids, label_ids), dim=1).to(device)
      fake_input = torch.cat((input_ids, out), dim=1).to(device)
      #print(label_ids)

      #discriminator 타겟 생성
      tgt_real = torch.ones((1, 1)).to(device).float()
      tgt_fake = torch.zeros((1, 1)).to(device).float()
      
      #real 역전파
      output_real = D(real_input)
      loss_real = criterion(output_real, tgt_real)
      loss_real.backward()
      optimizer.step()
      optimizer.zero_grad()
      #fake 역전파
      output_fake = D(fake_input)
      loss_fake = criterion(output_fake, tgt_fake)
      loss_fake.backward()
      optimizer.step()
      optimizer.zero_grad()

      out_real.append(output_real.cpu().item())
      out_fake.append(output_fake.cpu().item())

      if i % 100 == 0:
        print(epoch, i, output_real, output_fake)
        torch.save(D.state_dict(), 'D_lstm10_fc2.pkl')
      i += 1

## Discriminator 성능 평가


In [ ]:
model.eval()
out_real_test = []
out_fake_test = []
 
for batch in tqdm(test_dataset):
    batch = tuple(t.to(device) for t in batch)
    input_ids, position_ids, token_ids, label_ids, *_ = batch
    input_ids = input_ids[0].reshape(1, -1)
    label_ids = label_ids[0].reshape(1, -1)
      
    #모델 출력 생성 (out)
    input_ids = torch.cat((torch.tensor(cut_seq_to_eos(input_ids[0].tolist())).reshape(1, -1), torch.tensor([[EOS_ID]])), dim=1).to(device).long()

    out = model.generate(input_ids, pad_token_id=-1, max_length=1000, eos_token_id=EOS_ID, num_beams=5, num_return_sequences=1)
    out = out[0][input_ids.size(-1)+1:-1].reshape(1, -1)

    label_ids = label_ids[label_ids!=-1]
    label_ids = torch.tensor(cut_seq_to_eos(label_ids.tolist())).reshape(1, -1).to(device)

    #discriminator 입력 생성 (질문과 응답을 이어붙임)
    real_input = torch.cat((input_ids, label_ids), dim=1).to(device)
    fake_input = torch.cat((input_ids, out), dim=1).to(device)

    output_real = D(real_input)
    output_fake = D(fake_input)

    out_real_test.append(output_real.cpu().item())
    out_fake_test.append(output_fake.cpu().item())

In [61]:
out_real_test = np.array(out_real_test)
out_fake_test = np.array(out_fake_test)

np.save('out_real_test.npy', out_real_test)
np.save('out_fake_test.npy', out_fake_test)

In [62]:
np.count_nonzero(out_real_test>out_fake_test) / len(out_real_test)

0.82

## Text Generation

In [18]:
i = 0
model.eval()
input_ids = enc.encode(input(">> User:") + enc.eos_token, return_tensors='pt').to(device)
#input_ids = torch.cat((torch.tensor(cut_seq_to_eos(input_ids[0].tolist())).reshape(1, -1), torch.tensor([[EOS_ID]])), dim=1).to(device)
    
#Beam search 사용해 하나의 Source 문장마다 50개의 응답 생성
out = model.generate(input_ids, pad_token_id=-1, max_length=1000, eos_token_id=EOS_ID, num_beams=50, num_return_sequences=50, top_k=200, temperature=0.9, no_repeat_ngram_size=3)
out_greedy = model.generate(input_ids, pad_token_id=-1, max_length=1000, eos_token_id=EOS_ID)
out_greedy = out_greedy[0][out_greedy[0]!=-1][input_ids.size(-1):-1]
input_ids = input_ids[0][:-1]

#print("Input : ", enc.decode(input_ids))
dis_score = np.array([])
for j in range(50):
    output = out[j]
    output = output[output!=-1][input_ids.size(-1)+1:-1]
    dis_score = np.append(dis_score, D(output.reshape(1, -1)).item())

rank = np.argsort(dis_score)[::-1]
for k, r in enumerate(rank):
    output = out[r]
    output = output[output!=-1][input_ids.size(-1)+1:-1]
    print("Beam+Discriminator Output {} : ".format(k+1), enc.decode(output), "Score : {:.4}".format(dis_score[r]))z

print("Greedy Output : ", enc.decode(out_greedy))

>> User:What's your favorite movie?
Beam+Discriminator Output 1 :  The Big Lebowski. Score : 0.5781
Beam+Discriminator Output 2 :  The Big Lebowski Score : 0.4769
Beam+Discriminator Output 3 :  I don't watch a lot of movies, so I don't really have one. Score : 0.3901
Beam+Discriminator Output 4 :  I don't really have a favorite. I like a lot of movies. Score : 0.361
Beam+Discriminator Output 5 :  I don't have one! Score : 0.3121
Beam+Discriminator Output 6 :  I don't really watch a lot of movies. Score : 0.3032
Beam+Discriminator Output 7 :  Eternal Sunshine of the Spotless Mind! Score : 0.3009
Beam+Discriminator Output 8 :  Blade Runner Score : 0.2968
Beam+Discriminator Output 9 :  Star Wars Episode IV : A New Hope Score : 0.2957
Beam+Discriminator Output 10 :  Pulp Fiction Score : 0.2936
Beam+Discriminator Output 11 :  I don't watch a lot of movies. Score : 0.2852
Beam+Discriminator Output 12 :  I don't really like movies. Score : 0.2842
Beam+Discriminator Output 13 :  Space Jam Scor

## Benchmark


### BLEU Score

In [ ]:
#BLEU Score 계산산
from torchmetrics import BLEUScore
metric = BLEUScore()
model.eval()
out_real_test = []
out_fake_test = []
score_sum = 0

for batch in tqdm(test_dataset):
    batch = tuple(t.to(device) for t in batch)
    input_ids, position_ids, token_ids, label_ids, *_ = batch
    input_ids = input_ids[0].reshape(1, -1)
    label_ids = label_ids[0].reshape(1, -1)
      
    #모델 출력 생성 (out)
    input_ids = torch.cat((torch.tensor(cut_seq_to_eos(input_ids[0].tolist())).reshape(1, -1), torch.tensor([[EOS_ID]])), dim=1).to(device).long()

    out = model.generate(input_ids, pad_token_id=-1, max_length=1000, eos_token_id=EOS_ID, num_beams=50, num_return_sequences=50, top_k=200, temperature=0.9, no_repeat_ngram_size=3)

    label_ids = label_ids[label_ids!=-1]
    label_ids = torch.tensor(cut_seq_to_eos(label_ids.tolist())).reshape(1, -1).numpy()
    
    dis_score = np.array([])
    for j in range(50):
        output = out[j]
        output = output[output!=-1][input_ids.size(-1)+1:-1]
        dis_score = np.append(dis_score, D(output.reshape(1, -1)).item())
    rank = np.argsort(dis_score)[::-1]
    output = out[rank[0]]
    output = output[output!=-1][input_ids.size(-1):-1]

    label = []
    generated = []
    for tok in label_ids[0]:
        label.append(enc.decode(tok))
    for tok in output:
        generated.append(enc.decode(tok))
    score = metric(label, generated)
    score_sum += score


### Diversity

In [14]:
# Diversity 평가

from collections import OrderedDict, defaultdict
import numpy as np

def prepare_for_bleu(sentence):
    sent = [x for x in sentence if x!=0]
    while len(sent)<4:
        sent.append('0')
    sent = ' '.join([str(x) for x in sent])
    return sent   

def cal_entropy(generated):
    # print 'in BLEU score calculation'
    # the maximum is bigram, so assign the weight into 2 half.
    etp_score = [0.0, 0.0, 0.0, 0.0]
    div_score = [0.0, 0.0, 0.0, 0.0]
    counter = [defaultdict(int), defaultdict(int), defaultdict(int), defaultdict(int)]
    for gg in generated:
        g = gg.rstrip('2').split()
        for n in range(4):
            for idx in range(len(g)-n):
                ngram = ' '.join(g[idx:idx+n+1])
                counter[n][ngram] += 1
    for n in range(4):
        total = sum(counter[n].values()) +1e-10
        for v in counter[n].values():
            etp_score[n] += - (v+0.0) /total * (np.log(v+0.0) - np.log(total))
        div_score[n] = (len(counter[n].values())+0.0) /total
    return etp_score, div_score

cal_entropy([prepare_for_bleu(s) for s in  res_all])

([2.8097373165958293,
  3.938952636841678,
  3.8874845719179723,
  3.5218412405832353],
 [0.24175824175797608,
  0.8082191780810846,
  0.9272727272710413,
  0.9459459459433893])

In [20]:
etp_scores = []
div_scores = []

for batch in tqdm(test_dataset):
    batch = tuple(t.to(device) for t in batch)
    input_ids, position_ids, token_ids, label_ids, *_ = batch
    input_ids = input_ids[0].reshape(1, -1)
    label_ids = label_ids[0].reshape(1, -1)
      
    #모델 출력 생성 (out)
    input_ids = torch.cat((torch.tensor(cut_seq_to_eos(input_ids[0].tolist())).reshape(1, -1), torch.tensor([[EOS_ID]])), dim=1).to(device).long()

    out = model.generate(input_ids, pad_token_id=-1, max_length=1000, eos_token_id=EOS_ID, num_beams=50, num_return_sequences=50, top_k=200, temperature=0.9, no_repeat_ngram_size=3)

    label_ids = label_ids[label_ids!=-1]
    label_ids = torch.tensor(cut_seq_to_eos(label_ids.tolist())).reshape(1, -1).numpy()
    
    dis_score = np.array([])
    for j in range(50):
        output = out[j]
        output = output[output!=-1][input_ids.size(-1)+1:-1]
        if len(output) > 0:
            dis_score = np.append(dis_score, D(output.reshape(1, -1)).item())
    rank = np.argsort(dis_score)[::-1]
    output = out[rank[0]]
    output = output[output!=-1][input_ids.size(-1):-1]

    label = []
    generated = []
    for tok in label_ids[0]:
        label.append(enc.decode(tok))
    for tok in output:
        generated.append(enc.decode(tok))
    
    etp, div = cal_entropy([prepare_for_bleu(s) for s in generated])
    etp_scores.append(etp)
    div_scores.append(div)

100%|██████████| 1000/1000 [36:56<00:00,  2.22s/it]


In [32]:
etp_scores, div_scores = np.array(etp_scores), np.array(div_scores)
print("Mean ETP Scores of Beam Search + Discriminator method")
print(etp_scores.mean(axis=0))
print("\n")
print("Mean DIV Scores of Beam Search + Discriminator method")
print(div_scores.mean(axis=0))

Mean ETP Scores of Beam Search + Discriminator method
[2.60093481 3.16976956 3.09910681 2.53060631]


Mean DIV Scores of Beam Search + Discriminator method
[0.373178   0.7281995  0.87249993 0.95076622]


In [27]:
etp_scores_greedy = []
div_scores_greedy = []

for batch in tqdm(test_dataset):
    batch = tuple(t.to(device) for t in batch)
    input_ids, position_ids, token_ids, label_ids, *_ = batch
    input_ids = input_ids[0].reshape(1, -1)
    label_ids = label_ids[0].reshape(1, -1)
      
    #모델 출력 생성 (out)
    input_ids = torch.cat((torch.tensor(cut_seq_to_eos(input_ids[0].tolist())).reshape(1, -1), torch.tensor([[EOS_ID]])), dim=1).to(device).long()

    out = model.generate(input_ids, pad_token_id=-1, max_length=1000, eos_token_id=EOS_ID, top_k=200, temperature=0.9, no_repeat_ngram_size=3)

    label_ids = label_ids[label_ids!=-1]
    label_ids = torch.tensor(cut_seq_to_eos(label_ids.tolist())).reshape(1, -1).numpy()
    
    output = out[0]
    output = output[output!=-1][input_ids.size(-1)+1:-1]

    rank = np.argsort(dis_score)[::-1]
    output = out[rank[0]]
    output = output[output!=-1][input_ids.size(-1):-1]

    label = []
    generated = []
    for tok in label_ids[0]:
        label.append(enc.decode(tok))
    for tok in output:
        generated.append(enc.decode(tok))
    
    etp, div = cal_entropy([prepare_for_bleu(s) for s in generated])
    etp_scores_greedy.append(etp)
    div_scores_greedy.append(div)

100%|██████████| 1000/1000 [05:18<00:00,  3.14it/s]


In [33]:
etp_scores_greedy, div_scores_greedy = np.array(etp_scores_greedy), np.array(div_scores_greedy)
print("Mean ETP Scores of Greedy method")
print(etp_scores_greedy.mean(axis=0))
print("\n")
print("Mean DIV Scores of Greedy method")
print(div_scores_greedy.mean(axis=0))

Mean ETP Scores of Greedy method
[2.52981549 3.04879787 2.97176381 2.37611847]


Mean DIV Scores of Greedy method
[0.40057563 0.73379508 0.87307077 0.95227216]
